In [1]:
import sys

import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict
import numpy as np
import datetime as dt
import geopandas as gpd
import dendropy
import csv
from shapely.geometry import Point
import matplotlib.path as mpath
import matplotlib.patches as mpatches
from pyproj import Proj,transform
import tqdm
import pandas as pd
from matplotlib import cm
import matplotlib
import matplotlib.ticker as plticker
import os
from math import radians, cos, sin, asin, sqrt




In [2]:
uk_map = gpd.read_file("../../Data/UK_geog_data/UK_map.json")
uk_map = uk_map.to_crs("EPSG:3395")
england = uk_map.loc[uk_map["NAME_1"] == "England"]

utla_map = gpd.read_file("../../Data/UK_geog_data/UTLA_administrative_areas.json")
utla_map = utla_map.to_crs("EPSG:3395")
england_utla = utla_map.loc[utla_map['CODE'].str.startswith("E")]

pc_map = gpd.read_file("../../Data/UK_geog_data/England_postcode_districts.json")
pc_map.crs = "epsg:27700"
pc_map = pc_map.to_crs("epsg:3395")

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3395')



/usr/local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [4]:
def true_date(date):
    
    year = str(date).split(".")[0]
    decimal = f'0.{str(date).split(".")[1]}'
    
    if year == '2020':
        day_no = float(decimal) * 366
    else:
        day_no = float(decimal) * 365
        
    delta = dt.timedelta(day_no)
    
    start = dt.date(year=int(year), month=1, day=1)
    tr_date = start + delta
    
    return str(tr_date)

def decimal_date(date):
    
    if type(date) == str:
        date = dt.datetime.strptime(date,"%Y-%m-%d").date()
    year = date.year
    if year == 2020:
        div = 366
    else:
        div = 365
    start = dt.date(year=year, month=1, day=1)
    decimal = (date - start).days/div
    
    dec_date = year + decimal
    return dec_date

## Process MCC trees

In [5]:
def make_mcc_tree_file(mcc_file, lineage_name,most_recent_sample,outdir):

    tree = dendropy.Tree.get(path=mcc_file, schema="nexus")
    
    #first, name all of the nodes - currently all internal nodes don't have name
    count = 0
    for node in tree.preorder_node_iter():
        count += 1
        node.label = count
    
    #then extract info from the tree
    #NB head is child of tail, head is end going forwards in time
    edge_dict = defaultdict(dict)
    write_file = os.path.join(outdir, f'{lineage_name}_mcc_data.csv')
    with open(write_file, 'w') as fw:
        writer = csv.DictWriter(fw, fieldnames=["node1","node2","length","start_long_3395","start_lat_3395","end_lat_3395", "end_long_3395", "start_lat_4326", "start_long_4326","end_long_4326","end_lat_4326", "start_adm2", "end_adm2", "start_pc", "end_pc", "start_utla", "end_utla", "start_code", "end_code", "start_dec_date", "end_dec_date", "start_real_date", "end_real_date"])
        writer.writeheader()
        
        count = 0
        for edge in tqdm.tqdm(tree.postorder_edge_iter()):
            write_dict = {}
            if edge.tail_node:
                count += 1
                edge_dict[count] = {}
                node1 = edge.tail_node
                node2 = edge.head_node
                length = edge.length
                original_start_lat = node1.annotations['coordinates1'].value
                original_start_long = node1.annotations['coordinates2'].value
                original_end_lat = node2.annotations['coordinates1'].value
                original_end_long = node2.annotations['coordinates2'].value
                start_dec_date = most_recent_sample - float(node1.annotations['height'].value)
                end_dec_date = most_recent_sample - float(node2.annotations['height'].value)
                start_real_date = true_date(start_dec_date)
                end_real_date = true_date(end_dec_date)
                
                start_long,start_lat = transform(inProj,outProj, original_start_long, original_start_lat)
                end_long, end_lat = transform(inProj,outProj, original_end_long, original_end_lat)
                
                start_loc = None
                end_loc = None
                start_point = Point(float(start_long),float(start_lat))
                end_point = Point(float(end_long),float(end_lat))
                for location, polygon in zip(england["Multi_loc"], england["geometry"]):
                    if polygon.contains(start_point):
                        start_loc = location
                    if polygon.contains(end_point):
                        end_loc = location
                    if start_loc and end_loc:
                        break
                        
                start_pc = None
                end_pc = None
                for location, polygon in zip(pc_map["PostDist"], pc_map["geometry"]):
                    if polygon.contains(start_point):
                        start_pc = location
                    if polygon.contains(end_point):
                        end_pc = location
                    if start_pc and end_pc:
                        break
                        
                start_utla = None
                end_utla = None
                start_code = None
                end_code = None
                for location, polygon, code in zip(england_utla["FILE_NAME"], england_utla["geometry"], england_utla["CODE"]):
                    if polygon.contains(start_point):
                        start_utla = location
                        start_code = code
                    if polygon.contains(end_point):
                        end_utla = location
                        end_code = code
                    if start_utla and end_utla:
                        break
                        
                if start_utla == "GREATER_LONDON_AUTHORITY":
                    start_code="E13000001|E13000002"
                if end_utla == "GREATER_LONDON_AUTHORITY":
                    end_code="E13000001|E13000002"
                

                write_dict["node1"] = node1.label
                write_dict['node2'] = node2.label
                write_dict["length"] = length
                write_dict["start_lat_3395"] = start_lat
                write_dict["start_long_3395"] = start_long
                write_dict["end_lat_3395"] = end_lat
                write_dict["end_long_3395"] = end_long
                write_dict["start_lat_4326"] = original_start_lat
                write_dict["start_long_4326"] = original_start_long
                write_dict["end_lat_4326"] = original_end_lat
                write_dict["end_long_4326"] = original_end_long
                write_dict["start_adm2"] = start_loc
                write_dict["end_adm2"] = end_loc
                write_dict["start_pc"] = start_pc
                write_dict["end_pc"] = end_pc
                write_dict["start_dec_date"] = start_dec_date
                write_dict["end_dec_date"] = end_dec_date
                write_dict["start_real_date"] = start_real_date
                write_dict["end_real_date"] = end_real_date
                write_dict["start_utla"] = start_utla
                write_dict["end_utla"] = end_utla
                write_dict["start_code"] = start_code
                write_dict["end_code"] = end_code
                
                writer.writerow(write_dict)

                
def get_most_recent_tips(mrts, input_dir):
    for file in tqdm.tqdm(os.listdir(input_dir)):
        if file.endswith("csv"):
            name = file.strip(".csv")
            dates = []
            with open(os.path.join(input_dir,file)) as f:
                data = csv.DictReader(f)
                for l in data:
                    seq_name = l['name']
                    date = decimal_date(seq_name.split("|")[1])
                    dates.append(date)
                    
            mrt = max(dates)
            mrts[name] = mrt
    return mrts
          

In [6]:
mrts = {}
mrts = get_most_recent_tips(mrts, "../input_files/small_lineages/jclusterfunk_lists/")
mrts = get_most_recent_tips(mrts,"../input_files/large_lineages/")

100%|██████████| 21/21 [00:00<00:00, 29.86it/s]


In [9]:
unwanted_mccs = ["0_64", "1_218", "2_437", "0_74", "2_431", "0_204", "0_316", "0_75", "1_357", "1_460", "0_421", "0_266", "0_335", "0_342", "0_381", "0_103", "0_355", "1_10", "0_260", "0_372", "0_261", "0_34"]
#these did not converge properly due to lack of data and so have been removed from the analysis

overall_dict = defaultdict(dict)
for mcc_file in os.listdir("../results/small_lineages/mcc_trees/"):
    if mcc_file.endswith(".mcc"):
        name = mcc_file.strip(".mcc")
        print(name)
        if name not in unwanted_mccs:
            mcc_file = f'../results/small_lineages/mcc_trees/{mcc_file}'
            make_mcc_tree_file(mcc_file, name,mrts[name], "../results/MCC_files_all/")


0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
1it [00:00,  5.66it/s]

0_101


11it [00:01,  6.71it/s]
1it [00:00,  7.05it/s]

0_103
0_105


11it [00:01,  7.53it/s]
0it [00:00, ?it/s]

...

43it [00:06,  6.56it/s]


2_88


799it [02:08,  6.24it/s]
1it [00:00,  6.64it/s]

2_89


27it [00:04,  6.59it/s]


In [ ]:
for mcc_file in os.listdir("../results/large_lineages/"):
    if mcc_file.endswith(".mcc"):
        name = mcc_file.strip(".mcc")
        print(name)
        mcc_file = f'../results/large_lineages/{mcc_file}'
        make_mcc_tree_file(mcc_file, name, mrts[name], "../results/MCC_files_all/")

0it [00:00, ?it/s]/var/folders/pd/bjtp4xt916vfpszlps7rr5_c0000gn/T/ipykernel_7754/3910382583.py:37: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  start_long,start_lat = transform(inProj,outProj, original_start_long, original_start_lat)
/var/folders/pd/bjtp4xt916vfpszlps7rr5_c0000gn/T/ipykernel_7754/3910382583.py:38: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  end_long, end_lat = transform(inProj,outProj, original_end_long, original_end_lat)
1336it [00:44, 28.90it/s]

## Process distances

In [10]:
def calc_distance(long1,lat1,long2,lat2):
    
     
    # radians which converts from degrees to radians.
    long1 = radians(long1)
    long2 = radians(long2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlong = long2 - long1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlong / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)
     

In [11]:
#to check the function works like I"m expecting
london = (-0.1257,51.51)
manchester = (-2.244, 53.484)

print(calc_distance(london[0], london[1], manchester[0], manchester[1]))

262.162946805854


In [12]:
for file in os.listdir("../results/MCC_files_all/"):
    if file.endswith(".csv"):
        name = "_".join(file.strip(".csv").split("_")[0:2])
        
        fw = open(f"../results/movements/{name}_movements.csv", 'w')
        fw.write(f"date,distance\n")
        with open(os.path.join("../results/MCC_files_all/", file)) as f:
            data = csv.DictReader(f)
            for line in data:
                if line["start_pc"] != line["end_pc"]:
            
                    start = (float(line['start_long_4326']),float(line["start_lat_4326"]))
                    end = (float(line['end_long_4326']),float(line["end_lat_4326"]))

                    distance = calc_distance(start[0],start[1],end[0],end[1])
                    
                    fw.write(f'{line["start_real_date"]},{distance}\n')
                    
        fw.close()
                    
                    
